In [7]:
import requests
import pandas as pd
import numpy as np

response_API = requests.get('https://api.openalex.org')
response_API.status_code # 200 means success

200

In [59]:
# PaperAuthorAffiliations


# 333,457 affiliated works for University of Washington

# https://api.openalex.org/works?filter=institutions.id:I201448701


# last institution {} issue:
# https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlszMiwgJ2h0dHBzOi8vb3BlbmFsZXgub3JnL1czMTg1MzQ4NTg4J10i

# author id issue:


UW_works_url = "https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils3LCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzQyMjUxNTk3NjcnXSI="


UW_works = requests.get(UW_works_url).json()

rows = []
count = 0
for r in UW_works['results']:
    
    if 'id' in r:
        paper_id = r['id'].rsplit('/W', 1)[-1]
    else: 
        paper_id = np.nan

    
    for a in r['authorships']:
        count += 1
        if 'id' in a['author']:
            author_id = a['author']['id'].rsplit('/A', 1)[-1]
        else:
            author_id = np.nan
            
        if 'display_name' in a['author']:
            author_display_name = a['author']['display_name']
        else:
            author_display_name = np.nan
        
        if 'author_position' in a:
            author_position = a['author_position']
        else:
            author_position = np.nan

        
        # get the second last instition if last_instition is an empty dictionary {}        
        institutions_list = a['institutions']
        
        if len(institutions_list) != 0 and isinstance(institutions_list[-1], dict) and not bool(institutions_list[-1]):
            institutions_list.pop()
                    
        if len(institutions_list) != 0:
            last_instition = institutions_list[-1:][0]
            
            if 'id' in last_instition and last_instition['id'] != None:
                aff_id = last_instition['id'].rsplit('/I', 1)[-1]
            else:
                aff_id = np.nan
            
            if 'display_name' in last_instition:
                aff_display_name = last_instition['display_name']
            else:
                aff_display_name = np.nan
            
            if 'country_code' in last_instition:
                country_code = last_instition['country_code']
            else:
                country_code = np.nan
            
        else:
            aff_id = np.nan
            display_name = np.nan
            contry_code = np.nan
    

        rows.append([paper_id, author_id, aff_id, aff_display_name, country_code, 
                     author_position, author_display_name])
    
df = pd.DataFrame(rows, columns=["Paper_id", "Author_ID", "Affiliation_ID", 
                                          "Affiliation_name", "Country_code", 
                                          "Author_sequence_number", "Author_name"])
df

,Paper_id,Author_ID,Affiliation_ID,Affiliation_name,Country_code,Author_sequence_number,Author_name
0,4225286463,2952135929,114027177,University of North Carolina at Chapel Hill,US,first,Le Huang
1,4225286463,2587329117,114027177,University of North Carolina at Chapel Hill,US,middle,Jonathan D. Rosen
2,4225286463,2279220919,114027177,University of North Carolina at Chapel Hill,US,middle,Quan Sun
3,4225286463,2111358758,114027177,University of North Carolina at Chapel Hill,US,middle,Jiawen Chen
4,4225286463,2149429328,201448701,University of Washington,US,middle,Marsha M. Wheeler
...,...,...,...,...,...,...,...
333,4237326040,1986295364,NaN,"Department of Materials Science & Engineering,...",None,middle,Stephanie J. Benight
334,4237326040,629959257,NaN,"Department of Materials Science & Engineering,...",None,middle,Ilya Kosilkin
335,4237326040,2162628756,NaN,"Department of Materials Science & Engineering,...",None,middle,Bruce G. Robinson
336,4237326040,2136157913,NaN,"Department of Materials Science & Engineering,...",None,middle,Bruce E. Eichinger


In [61]:
UW_works_url_with_cursor = 'https://api.openalex.org/works?filter=institutions.id:I201448701&cursor={}'


cursor = '*'
PaperAuthorAffiliations_rows = []

count = 1

# loop through pages
while cursor:
    UW_works_url = UW_works_url_with_cursor.format(cursor)
    print("\n" + str(count) + ": " + UW_works_url)

    UW_works = requests.get(UW_works_url).json()
        
    for r in UW_works['results']:
        if 'id' in r:
            paper_id = r['id'].rsplit('/W', 1)[-1]
        else: 
            paper_id = np.nan


        for a in r['authorships']:
            if 'id' in a['author']:
                author_id = a['author']['id'].rsplit('/A', 1)[-1]
            else:
                author_id = np.nan

            if 'display_name' in a['author']:
                author_display_name = a['author']['display_name']
            else:
                author_display_name = np.nan

            if 'author_position' in a:
                author_position = a['author_position']
            else:
                author_position = np.nan


            # get the second last instition if last_instition is an empty dictionary {}        
            institutions_list = a['institutions']
            if len(institutions_list) != 0 and isinstance(institutions_list[-1], dict) and not bool(institutions_list[-1]):
                institutions_list.pop()

            if len(institutions_list) != 0:
                last_instition = institutions_list[-1:][0]

                if 'id' in last_instition and last_instition['id'] != None:
                    aff_id = last_instition['id'].rsplit('/I', 1)[-1]
                else:
                    aff_id = np.nan

                if 'display_name' in last_instition:
                    aff_display_name = last_instition['display_name']
                else:
                    aff_display_name = np.nan

                if 'country_code' in last_instition:
                    country_code = last_instition['country_code']
                else:
                    country_code = np.nan

            else:
                aff_id = np.nan
                display_name = np.nan
                contry_code = np.nan


            PaperAuthorAffiliations_rows.append([paper_id, author_id, aff_id, 
                                                 aff_display_name, country_code, 
                                                 author_position, author_display_name])

    # update cursor to meta.next_cursor
    cursor = UW_works['meta']['next_cursor']
    
    count += 1


PaperAuthorAffiliations_df = pd.DataFrame(PaperAuthorAffiliations_rows, 
                                          columns=["Paper_id", "Author_ID", "Affiliation_ID", 
                                          "Affiliation_name", "Country_code", 
                                          "Author_sequence_number", "Author_name"])
PaperAuthorAffiliations_df


1: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=*

2: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils3ODQyLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzIxNTc5MTc0MTEnXSI=

3: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils1Mjc5LCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzIwOTYxMTMyMzYnXSI=

4: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils0NDMyLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzIwNjUzMDQzNTMnXSI=

5: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils0MDI2LCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzIxNzQ2NjE3NDknXSI=

6: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlszNDE0LCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzIwMTE4MzI5NjInXSI=

7: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlszMTkyLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzIxNjQwNzU2NDYnXSI=

8: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlsyODE4LCAnaHR0cHM6Ly9v

KeyboardInterrupt: 